In [1]:
!pip install transformers
!pip install datasets evaluate accelerate optuna optimum
!pip install optimum[onnxruntime]@git+https://github.com/huggingface/optimum.git
!pip install onnxruntime-gpu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 78.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 29.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.2/404.2 kB 45.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.1/396.1 kB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import time
import shutil
import torch
import pandas as pd
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import Trainer, TrainingArguments
from datasets import Features, Value, ClassLabel
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from transformers import AutoModelForSequenceClassification, AutoConfig
from optimum.onnxruntime import ORTQuantizer, ORTModelForSequenceClassification
from optimum.onnxruntime.configuration import AutoQuantizationConfig

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [3]:
df = pd.read_csv("/content/helpdest_dataset.csv",encoding='utf-8')

In [4]:
df.head()

,text,label
0,The customer described problems with blurry im...,2
1,The customer was having issues with login cred...,2
2,The customer complained that the product they ...,2
3,The customer asked about the status of their d...,0
4,The customer needed help connecting their new ...,2


In [5]:
def finetune(df):
  train , validation = train_test_split(df , test_size=0.09 , random_state = 42 , stratify = df["label"])
  train.to_csv("train_help.csv" , index=False)
  validation.to_csv("val_help.csv",index=False)

  class_names = ['Negative','Neutral','Positive']
  ft=Features({'text':Value(dtype='string',id=None), 'label': ClassLabel(num_classes=3, names=class_names)})
  dataset=load_dataset('csv',data_files={'train':'train_help.csv','validation':'val_help.csv'},features=ft)

  model_ckpt = "distilbert-base-uncased"
  tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

  def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

  helpdesk_encoded = dataset.map(tokenize, batched=True, batch_size=None)
  helpdesk_encoded.set_format("torch", columns=["input_ids", "attention_mask", "label"])

  device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

  num_labels = 3
  model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

  config = (AutoConfig.from_pretrained(model_ckpt,
                                    num_labels=len(class_names),
                                    id2label={i: label for i, label in enumerate(class_names)},
                                    label2id={label: i for i, label in enumerate(class_names)}))

  model.config = config


  def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

  batch_size = 2
  logging_steps = len(helpdesk_encoded["train"]) // batch_size
  model_name = "Venkatesh4342/distilbert-helpdesk-sentiment"
  training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=6,
    learning_rate=2e-5,
    evaluation_strategy ='steps',
    eval_steps=100,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_steps=100,
    save_total_limit=3,
    load_best_model_at_end= True,
    logging_steps=logging_steps,
    gradient_checkpointing=True,
    push_to_hub=True)


  trainer = Trainer(model=model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=helpdesk_encoded["train"],
                  eval_dataset=helpdesk_encoded["validation"],
                  tokenizer=tokenizer)

  trainer.train()
  trainer.save_model("fine_tuned_model")

  onnx_model = ORTModelForSequenceClassification.from_pretrained("fine_tuned_model", export=True)
  quantizer = ORTQuantizer.from_pretrained(onnx_model)
  dqconfig = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)
  model_quantized_path = quantizer.quantize(
    save_dir="model",
    quantization_config=dqconfig,
  )
  time.sleep(15)
  api.upload_folder(
    folder_path="model",
    repo_id="Venkatesh4342/quantized-helpdesk",
    repo_type="model"
)
  shutil.rmtree("fine_tuned_model")
  os.remove("train_help.csv")
  os.remove("val_help.csv")

In [6]:
finetune(df)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/90 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Accuracy,F1
100,No log,0.371040,0.888889,0.888190
200,No log,0.204870,0.922222,0.921342
300,No log,0.112782,0.977778,0.977775
400,No log,0.277320,0.911111,0.908428
500,0.327400,0.113064,0.955556,0.955364
600,0.327400,0.087544,0.988889,0.988886


Framework not specified. Using pt to export to ONNX.
Using the export variant default. Available variants are:
	- default: The default ONNX variant.
Using framework PyTorch: 2.0.1+cu118
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:223: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: model (external data format: False)
Configuration saved in model/ort_config.json
